# Creating the local DB and csv file with all trials data
This notebooks shows how to create static copies of CLinicalTrials.goc databases from manually downloaded json clinical trial records.
The created sqlite database and csv file can be later used in the analysis pipelines.


In [1]:
# Imports
import requests
import json
import pandas as pd
from tqdm import tqdm
from datetime import datetime as dt

import requests
import xml.etree.ElementTree as ET
import os
import json
import sqlite3
from typing import Any


# Download all records locally
While not the most efficient, due to clinicaltrials.gov API limitations (max 1000 records per query), this might be the best way to proceed
So.. first we need to download the full dataset from: 
    
    https://classic.clinicaltrials.gov/api/gui/ref/download_all

We chose the JSON download format:
    https://classic.clinicaltrials.gov/AllAPIJSON.zip
## Check what study fields are avaialble


In [2]:
print(f"{dt.now().strftime('%Y-%m-%d %H:%M:%S')} - Start")
all_study_fields_url ="https://classic.clinicaltrials.gov/api/info/study_fields_list"
response = requests.get(all_study_fields_url)
all_study_fields_xml  = response.text

# Parse the XML data
root = ET.fromstring(all_study_fields_xml)

# Find all Field elements within FieldList
field_elements = root.findall(".//FieldList/Field")

# Extract the Field Name attribute and store in a list
all_study_fields = [str(field.get("Name")) for field in field_elements]

# Print the list of field names
print(all_study_fields)
print(f"{len(all_study_fields)} fields in total")
print(f"{dt.now().strftime('%Y-%m-%d %H:%M:%S')} - End")

2023-10-19 21:08:52 - Start
['Acronym', 'AgreementOtherDetails', 'AgreementPISponsorEmployee', 'AgreementRestrictionType', 'AgreementRestrictiveAgreement', 'ArmGroupDescription', 'ArmGroupInterventionName', 'ArmGroupLabel', 'ArmGroupType', 'AvailIPDComment', 'AvailIPDId', 'AvailIPDType', 'AvailIPDURL', 'BaselineCategoryTitle', 'BaselineClassDenomCountGroupId', 'BaselineClassDenomCountValue', 'BaselineClassDenomUnits', 'BaselineClassTitle', 'BaselineDenomCountGroupId', 'BaselineDenomCountValue', 'BaselineDenomUnits', 'BaselineGroupDescription', 'BaselineGroupId', 'BaselineGroupTitle', 'BaselineMeasureCalculatePct', 'BaselineMeasureDenomCountGroupId', 'BaselineMeasureDenomCountValue', 'BaselineMeasureDenomUnits', 'BaselineMeasureDenomUnitsSelected', 'BaselineMeasureDescription', 'BaselineMeasureDispersionType', 'BaselineMeasureParamType', 'BaselineMeasurePopulationDescription', 'BaselineMeasureTitle', 'BaselineMeasureUnitOfMeasure', 'BaselineMeasurementComment', 'BaselineMeasurementGroup

## Iterative scan per json into a dataframe
After we download over 10GB of data, we can parse it into a csv file/sql database or anything similar that we can then use in downstream calculations


In [3]:
# Setup

# Create an empty list to store the data records
data_records = []
max_studies = 10000000 # <-- set to a smaller number for testing. There are less than 0.5m studies in total as of Oct 2023
n_write = 100000 # write to database every n_write records, larger values are faster but use more memory.
# selected_study_fields = ['NCTId', 'Condition', 'BriefTitle'] # <-- smaller set of fields for testing
selected_study_fields = all_study_fields


# Specify the top-level folder path containing subfolders with JSON files
folder_path = '../data/AllAPIJSON/'

rebuild_db = True # <-- set to True to rebuild the database from scratch
only_add_new_records = True # <-- set to True to add new records to the database instead of rebuilding all records

In [4]:
# some potetnially useful functions



def convert_lists_to_strings(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convert list values in a DataFrame to strings.

    Parameters:
    df (pd.DataFrame): Input DataFrame containing columns with list values.

    Returns:
    pd.DataFrame: DataFrame with list values converted to strings.
    """
    # Iterate over the DataFrame's columns
    for col in df.columns:
        # Check if the column contains lists
        if df[col].apply(lambda x: isinstance(x, list)).any():
            # Convert lists to strings
            df[col] = df[col].astype(str)
    return df

def extract_field(data: Any, field_name: str) -> Any:
    """
    Extract a specific field value from nested dictionaries or lists within data.

    Parameters:
    data (dict or list): The data structure to search for the field.
    field_name (str): The name of the field to extract.

    Returns:
    Any: The extracted field value or None if not found.
    """
    if isinstance(data, dict):
        if field_name in data:
            return data[field_name]
        else:
            for key, value in data.items():
                result = extract_field(value, field_name)
                if result is not None:
                    return result
    elif isinstance(data, list):
        for item in data:
            result = extract_field(item, field_name)
            if result is not None:
                return result
    return None



In [6]:
if rebuild_db:
    table_name = 'alltrials' # In inital run I was not sure what goes into the 'table_name' and was putting the path '../data/alltrials' instead of just 'alltrials'. 
    print(f"{dt.now().strftime('%Y-%m-%d %H:%M:%S')} - Rebuilding database")
    conn = sqlite3.connect('../data/alltrials.db')  
    existing_nctids = set()  # Create a set to store existing NCTIDs

    if only_add_new_records:
        
        # Load existing NCTIDs from the SQL database
        cursor = conn.cursor()
        cursor.execute(f"SELECT NCTid FROM '{table_name}' ")
        existing_nctids.update(row[0] for row in cursor.fetchall())

    i = 0 # Counter for files processed
    for subfolder_name in tqdm(os.listdir(folder_path)):
        subfolder_path = os.path.join(folder_path, subfolder_name)

        # Check if the item in the top folder is a directory (subfolder)
        if os.path.isdir(subfolder_path):    
            for filename in os.listdir(subfolder_path):
                if i > max_studies:
                    break
                if filename.endswith('.json'):
                    i+=1
                    nctid = filename.replace('.json', '')  # Extract NCTid from filename
                    if only_add_new_records and nctid in existing_nctids:
                        continue
                    else:
                        file_path = os.path.join(subfolder_path, filename)

                        # Read JSON data
                        with open(file_path, 'r') as json_file:
                            study_data = json.load(json_file)
                            row_data = [extract_field(study_data, field) for field in selected_study_fields]
                            data_records.append(row_data)
                if (i % n_write == 0): # save every n_write records and empty the list
                    df = pd.DataFrame(data_records)
                    df = convert_lists_to_strings(df)
                    df.columns = selected_study_fields
                    
                    df.to_csv('../data/alltrials.csv', mode='a', header=True, sep="\t")
                    df.to_sql(table_name, conn, if_exists='append', index=False)
                    data_records = []    
    # Final write of the remaining data after the loop
    if len(data_records) > 0:
        df = pd.DataFrame(data_records)
        df = convert_lists_to_strings(df)
        df.columns = selected_study_fields
        df.to_csv('../data/alltrials.csv', mode='a', header=True, sep="\t")
        df.to_sql(table_name, conn, if_exists='append', index=False)
    print(f"{dt.now().strftime('%Y-%m-%d %H:%M:%S')} - Finished rebuilding database")   
                    


2023-10-19 21:09:27 - Rebuilding database


100%|██████████| 611/611 [3:44:26<00:00, 22.04s/it]  


2023-10-20 00:54:53 - Finished rebuilding database
